In [ ]:
Data Cleaning
I have now been able to extract the data from the website even though I had some challenges,we can see that it is not cleaned and ready to be analyzed yet. The reviews section will need to be cleaned for punctuations, spellings and other characters.

In [1]:
#imports

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

#regex
import re

In [3]:
#create a dataframe from csv file
import os
import pandas as pd

file_path = "./BA_reviews.csv"  

if os.path.exists(file_path):
    df = pd.read_csv(file_path, index_col=0)
else:
    print(f"File not found at {file_path}. Please check the file path.")


In [4]:
df.head()

,reviews,stars,date,country
0,LHR to PHL on 787 online check in worked well ...,5.0,22nd September 2014,Austria
1,BHD-LHR. I flew BA from Belfast George Best Ai...,7.0,22nd September 2014,Egypt
2,Considering our outbound flight to Orlando was...,9.0,22nd September 2014,United Kingdom
3,LHR-SIN in Economy. Very relaxed and prompt ch...,1.0,22nd September 2014,United Kingdom
4,LHR-LIS in Club Europe (on Avios). Easy check-...,8.0,18th September 2014,United Kingdom


I will then be creating a column which mentions if the user is verified or not

In [5]:
df['verified'] = df.reviews.str.contains("Trip Verified")

In [6]:
df['verified']


0     False
1     False
2     False
3     False
4     False
      ...  
95    False
96    False
97    False
98    False
99    False
Name: verified, Length: 100, dtype: bool

Cleaning Reviews

I will then move on to extract the column of reviews into a separate dataframe and clean it for semantic analysis

In [14]:
import pandas as pd
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

# Import NLTK and download necessary data
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

# Initialize the WordNet Lemmatizer
lemma = WordNetLemmatizer()

# Read the original DataFrame from a CSV file
df = pd.read_csv("./BA_reviews.csv") 
# 1. Extract the 'reviews' column into a separate DataFrame
reviews_df = df[['reviews']]

# 2. Data cleaning and preprocessing
reviews_df['cleaned_reviews'] = reviews_df['reviews'].str.strip("✅ Trip Verified |")
corpus = []

for review in reviews_df['cleaned_reviews']:
    review = re.sub('[^a-zA-Z]', ' ', review)
    review = review.lower()
    review = review.split()
    review = [lemma.lemmatize(word) for word in review if word not in set(stopwords.words("english"))]
    review = " ".join(review)
    corpus.append(review)

reviews_df['cleaned_reviews'] = corpus

# At this point, 'reviews_df' contains the cleaned reviews in the 'cleaned_reviews' column.

# 3. Perform semantic analysis or any other text analysis as needed.
# You can use libraries like NLTK, spaCy, or others for further analysis.

# Example: Count the number of words in each cleaned review
reviews_df['word_count'] = reviews_df['cleaned_reviews'].apply(lambda x: len(x.split()))

# Continue with your text analysis or modeling as required.


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/esselkobby/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/esselkobby/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
/tmp/ipykernel_46821/852727407.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews_df['cleaned_reviews'] = reviews_df['reviews'].str.strip("✅ Trip Verified |")
/tmp/ipykernel_46821/852727407.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#retu

In [16]:
# add the corpus to the original dataframe

df['corpus'] = corpus

In [17]:

df.head()

,Unnamed: 0,reviews,stars,date,country,corpus
0,0,LHR to PHL on 787 online check in worked well ...,5.0,22nd September 2014,Austria,lhr phl online check worked well first class l...
1,1,BHD-LHR. I flew BA from Belfast George Best Ai...,7.0,22nd September 2014,Egypt,bhd lhr flew ba belfast george best airport lo...
2,2,Considering our outbound flight to Orlando was...,9.0,22nd September 2014,United Kingdom,considering outbound flight orlando delayed go...
3,3,LHR-SIN in Economy. Very relaxed and prompt ch...,1.0,22nd September 2014,United Kingdom,lhr sin economy relaxed prompt check good atte...
4,4,LHR-LIS in Club Europe (on Avios). Easy check-...,8.0,18th September 2014,United Kingdom,lhr li club europe avios easy check security b...


Cleaning/Format Date

In [19]:
df.dtypes

Unnamed: 0      int64
reviews        object
stars         float64
date           object
country        object
corpus         object
dtype: object

In [24]:
# convert the date to datetime format

df.date = pd.to_datetime(df.date)

ValueError: time data "18th September 2014" doesn't match format "%dnd %B %Y", at position 1. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [21]:
df.date.head()

0    22nd September 2014
1    22nd September 2014
2    22nd September 2014
3    22nd September 2014
4    18th September 2014
Name: date, dtype: object

Cleaning ratings with stars

In [25]:
#check for unique values
df.stars.unique()

array([ 5.,  7.,  9.,  1.,  8.,  2., 10.,  4.,  6.,  3., nan])

In [27]:

df.stars.value_counts()

stars
1.0     18
7.0     13
9.0     13
8.0     13
10.0    12
5.0      7
2.0      6
4.0      6
3.0      6
6.0      5
Name: count, dtype: int64

In [28]:


# drop the rows where the value of ratings is None
df.drop(df[df.stars == "None"].index, axis=0, inplace=True)



In [29]:


#check the unique values again
df.stars.unique()



array([ 5.,  7.,  9.,  1.,  8.,  2., 10.,  4.,  6.,  3., nan])

Checking for null Values

In [30]:


df.isnull().value_counts()



Unnamed: 0  reviews  stars  date   country  corpus
False       False    False  False  False    False     99
                     True   False  False    False      1
Name: count, dtype: int64

In [31]:

df.country.isnull().value_counts()


country
False    100
Name: count, dtype: int64

In [32]:
#drop the rows using index where the country value is null
df.drop(df[df.country.isnull() == True].index, axis=0, inplace=True)


In [33]:
df.shape

(100, 6)

In [34]:
#resetting the index
df.reset_index(drop=True)

,Unnamed: 0,reviews,stars,date,country,corpus
0,0,LHR to PHL on 787 online check in worked well ...,5.0,22nd September 2014,Austria,lhr phl online check worked well first class l...
1,1,BHD-LHR. I flew BA from Belfast George Best Ai...,7.0,22nd September 2014,Egypt,bhd lhr flew ba belfast george best airport lo...
2,2,Considering our outbound flight to Orlando was...,9.0,22nd September 2014,United Kingdom,considering outbound flight orlando delayed go...
3,3,LHR-SIN in Economy. Very relaxed and prompt ch...,1.0,22nd September 2014,United Kingdom,lhr sin economy relaxed prompt check good atte...
4,4,LHR-LIS in Club Europe (on Avios). Easy check-...,8.0,18th September 2014,United Kingdom,lhr li club europe avios easy check security b...
...,...,...,...,...,...,...
95,95,LHR-CPH-LHR Business Class. This is a joke. Sc...,4.0,31st July 2014,United Kingdom,lhr cph lhr business class joke scruffy aircra...
96,96,LHR-CDG-LHR. Outbound A320 on 26/7 return A319...,NaN,31st July 2014,United Kingdom,lhr cdg lhr outbound return euro traveller fli...
97,97,Toronto to London in WTP on a 787 and London t...,7.0,31st July 2014,Canada,oronto london wtp london toronto club world el...
98,98,12/2013. LHR-LAX. A380. Aircraft was clean. Se...,10.0,31st July 2014,Netherlands,lhr lax aircraft clean seat comfortable leg sp...


I am glad to come to this point where our data is cleaned and now ready for data visualization and data analysis 

In [16]:
import pandas as pd
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

# Import NLTK and download necessary data
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

# Initialize the WordNet Lemmatizer
lemma = WordNetLemmatizer()

# Read the original DataFrame from a CSV file
df = pd.read_csv("./BA_reviews.csv")  
# Perform data cleaning and preprocessing here

# Export the cleaned DataFrame to a CSV file
relative_export_path = "cleaned-BA-reviews.csv"
df.to_csv(relative_export_path)


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/esselkobby/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/esselkobby/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
